In [ ]:
import os
import matplotlib.pyplot as plt
import json
import matplotlib.image as mpimg
import numpy as np

rootdir = './test/'           

# Auxiliar functions

In [ ]:
def checkJson(json, name=None, initial_lr=None, dropout=None, l2_reg=None, batch_norm=None, data_aug=None):
    if name is not None and json["name_model"]!=name:
        return False
    if initial_lr is not None and json["initial_lr"]!=initial_lr:
        return False
    if dropout is not None and json["dropout"]!=dropout:
        return False
    if l2_reg is not None and json["l2_reg"]!=l2_reg:
        return False
    if batch_norm is not None and json["batch_norm"]!=batch_norm:
        return False
    if data_aug is not None and json["data_aug"]!=data_aug:
        return False
    return True
    
def confidenceInterval(accuracy, n_test):
    if accuracy==-100:
        return -1
    return 1.96*np.sqrt(accuracy*(100-accuracy)/n_test)

def getBestValAcc(data):
    try:
        return np.amax(np.array(list(data["train_result"]["val_accuracy"].values())))
    except:
        try: 
            return np.amax(np.array(list(data["train_result_augmentation"]["val_accuracy"].values())))
        except:
            return -1       

# MLP graphs

## No regularization

In [ ]:
header = "Learning Rate & "
row = "2 Hidden Layers & "
row4 = "4 Hidden Layers & "
row8 = "8 Hidden Layers & "

for subdir, dirs, files in os.walk(rootdir):
    for dir in sorted(dirs):
        if (dir!=".ipynb_checkpoints"):
            #print(rootdir+dir+"/experiment.json")
            f = open(rootdir+dir+"/experiment.json")
            data = json.load(f)
            if checkJson(data["parameters"], name="MLP-2layers"):
                header += "$"+str('{:.3f}'.format(data["parameters"]["initial_lr"])) + "$ & "
                row2+= "$"+str('{:.2f}'.format(getBestValAcc(data)*100))+"\pm"+str('{:.2f}'.format(confidenceInterval(getBestValAcc(data)*100, 10_000))) + "$ &"
            if checkJson(data["parameters"], name="MLP-4layers"):
                header += "$"+str('{:.3f}'.format(data["parameters"]["initial_lr"])) + "$ & "
                row4+= "$"+str('{:.2f}'.format(getBestValAcc(data)*100))+"\pm"+str('{:.2f}'.format(confidenceInterval(getBestValAcc(data)*100, 10_000))) + "$ &"
            if checkJson(data["parameters"], name="MLP-8layers"):
                header += "$"+str('{:.3f}'.format(data["parameters"]["initial_lr"])) + "$ & "
                row8+= "$"+str('{:.2f}'.format(getBestValAcc(data)*100))+"\pm"+str('{:.2f}'.format(confidenceInterval(getBestValAcc(data)*100, 10_000))) + "$ &"

                
print(header[0:-1])
print(row2[0:-1])
print(row4[0:-1])
print(row8[0:-1])

In [ ]:
name_experiments=["VGG16"]
info =""
for name_experiment in name_experiments:
    for subdir, dirs, files in os.walk(rootdir): 
        for dir in sorted(dirs):
            if (dir!=".ipynb_checkpoints") and os.path.exists(rootdir+dir+"/experiment.json"):
                f = open(rootdir+dir+"/experiment.json")
                data = json.load(f)
                
                if data["parameters"]["name_model"].find(name_experiment) != -1:
                    print(dir)
                    info = data["parameters"]["name_model"] +str(' {:.2f} \pm {:.2f}'.format(getBestValAcc(data)*100,confidenceInterval(getBestValAcc(data)*100, 10_000)))
                    print(info)            
                f.close()  
            


In [ ]:
fig, axs = plt.subplots(1, 2,figsize=(20,16))


for ax in axs:
    #for a in ax:
    ax.set_axis_off()

for subdir, dirs, files in os.walk(rootdir):
    for dir in sorted(dirs):
        if (dir!=".ipynb_checkpoints") and os.path.exists(rootdir+dir+"/experiment.json"):
            f = open(rootdir+dir+"/experiment.json")
            data = json.load(f)
            print(dir,data["parameters"]["name_model"], data["parameters"]["initial_lr"])
            if checkJson(data["parameters"], name="VGG16_all_transfer"):
                img = mpimg.imread(rootdir+dir+"/accuracy.png")
                axs[0].set_title("VGG 16 without frozen layers", fontsize="xx-large")
                axs[0].imshow(img)
            if checkJson(data["parameters"], name="VGG16_all_transfer_modtrue"):
                img = mpimg.imread(rootdir+dir+"/accuracy.png")
                axs[1].set_title("VGG 16 with frozen layers", fontsize="xx-large")
                axs[1].imshow(img)

In [ ]:
fig, axs = plt.subplots(1, 2,figsize=(20,16))


for ax in axs:
    #for a in ax:
    ax.set_axis_off()

for subdir, dirs, files in os.walk(rootdir):
    for dir in sorted(dirs):
        if (dir!=".ipynb_checkpoints") and os.path.exists(rootdir+dir+"/experiment.json"):
            f = open(rootdir+dir+"/experiment.json")
            data = json.load(f)
            print(dir,data["parameters"]["name_model"], data["parameters"]["initial_lr"])
            if checkJson(data["parameters"], name="VGG16_initial_lr_5e-2", initial_lr=0.05):
                img = mpimg.imread(rootdir+dir+"/accuracy.png")
                axs[0].set_title("VGGs 16 Lr (lambda=0.05)", fontsize="xx-large")
                axs[0].imshow(img)
            if checkJson(data["parameters"], name="VGG19_initial_lr_5e-2", initial_lr=0.05):
                img = mpimg.imread(rootdir+dir+"/accuracy.png")
                axs[1].set_title("VGGs 19 Lr (lambda=0.05)", fontsize="xx-large")
                axs[1].imshow(img)

In [ ]:
fig, axs = plt.subplots(3, 3,figsize=(20,16))


for ax in axs:
    for a in ax:
        a.set_axis_off()

for subdir, dirs, files in os.walk(rootdir):
    for dir in sorted(dirs):
        if (dir!=".ipynb_checkpoints") and os.path.exists(rootdir+dir+"/experiment.json"):
            f = open(rootdir+dir+"/experiment.json")
            data = json.load(f)
            print(dir,data["parameters"]["name_model"], data["parameters"]["initial_lr"])
            if checkJson(data["parameters"], name="VGG16_batch_norm", batch_norm=True):
                img = mpimg.imread(rootdir+dir+"/accuracy.png")
                axs[0,0].set_title("VGGs 16 Batch Norm", fontsize="xx-large")
                axs[0,0].imshow(img)
            if checkJson(data["parameters"], name="VGG19_batch_norm", batch_norm=True):
                img = mpimg.imread(rootdir+dir+"/accuracy.png")
                axs[0,1].set_title("VGGs 19 Batch Norm", fontsize="xx-large")
                axs[0,1].imshow(img)
            if checkJson(data["parameters"], name="VGG16_data_augmentation", data_aug=True):
                img = mpimg.imread(rootdir+dir+"/accuracy.png")
                axs[1,0].set_title("VGGs 19 Data augmentation ", fontsize="xx-large")
                axs[1,0].imshow(img)
            if checkJson(data["parameters"], name="VGG19_data_augmentation", data_aug=True):
                img = mpimg.imread(rootdir+dir+"/accuracy.png")
                axs[1,1].set_title("VGGs 19 Data augmentation", fontsize="xx-large")
                axs[1,1].imshow(img)
            if checkJson(data["parameters"], name="VGG16_all",batch_norm=True, data_aug=True):
                img = mpimg.imread(rootdir+dir+"/accuracy.png")
                axs[2,1].set_title("VGGs 16 all", fontsize="xx-large")
                axs[2,1].imshow(img)
            if checkJson(data["parameters"], name="VGG19_all",batch_norm=True, data_aug=True):
                img = mpimg.imread(rootdir+dir+"/accuracy.png")
                axs[2,1].set_title("VGGs 19 all", fontsize="xx-large")
                axs[2,1].imshow(img)

In [ ]:
header = "L2 regularization $\lambda$ & "
row2 = "2 Hidden Layers & "
row4 = "4 Hidden Layers & "
row8 = "8 Hidden Layers & "

for subdir, dirs, files in os.walk(rootdir):
    for dir in sorted(dirs):
        if (dir!=".ipynb_checkpoints"):
            #print(rootdir+dir+"/experiment.json")
            f = open(rootdir+dir+"/experiment.json")
            data = json.load(f)
            if checkJson(data["parameters"], name="MLP-2layers-reg"):
                # header += "$"+str('{:.3f}'.format(data["parameters"]["l2_reg"])) + "$ & "
                row2+= "$"+str('{:.2f}'.format(getBestValAcc(data)*100))+"\pm"+str('{:.2f}'.format(confidenceInterval(getBestValAcc(data)*100, 10_000))) + "$ &"
            if checkJson(data["parameters"], name="MLP-4layers-reg"):
                # header += "$"+str('{:.3f}'.format(data["parameters"]["l2_reg"])) + "$ & "
                row4+= "$"+str('{:.2f}'.format(getBestValAcc(data)*100))+"\pm"+str('{:.2f}'.format(confidenceInterval(getBestValAcc(data)*100, 10_000))) + "$ &"
            if checkJson(data["parameters"], name="MLP-8layers-reg"):
                # header += "$"+str('{:.3f}'.format(data["parameters"]["l2_reg"])) + "$ & "
                row8+= "$"+str('{:.2f}'.format(getBestValAcc(data)*100))+"\pm"+str('{:.2f}'.format(confidenceInterval(getBestValAcc(data)*100, 10_000))) + "$ &"
    
                
print(header[0:-1])
print(row2[0:-1]+ " \\")
print(row4[0:-1]+ " \\")
print(row8[0:-1]+ " \\")

In [ ]:
header = "Dropout & "
row2 = "2 Hidden Layers & "
row4 = "4 Hidden Layers & "
row8 = "8 Hidden Layers & "

for subdir, dirs, files in os.walk(rootdir):
    for dir in sorted(dirs):
        if (dir!=".ipynb_checkpoints"):
            #print(rootdir+dir+"/experiment.json")
            f = open(rootdir+dir+"/experiment.json")
            data = json.load(f)
            if checkJson(data["parameters"], name="MLP-2layers-dropout"):
                header += "$"+str('{:.3f}'.format(data["parameters"]["dropout"])) + "$ & "
                row2+= "$"+str('{:.2f}'.format(getBestValAcc(data)*100))+"\pm"+str('{:.2f}'.format(confidenceInterval(getBestValAcc(data)*100, 10_000))) + "$ &"
            if checkJson(data["parameters"], name="MLP-4layers-dropout"):
                # header += "$"+str('{:.3f}'.format(data["parameters"]["l2_reg"])) + "$ & "
                row4+= "$"+str('{:.2f}'.format(getBestValAcc(data)*100))+"\pm"+str('{:.2f}'.format(confidenceInterval(getBestValAcc(data)*100, 10_000))) + "$ &"
            if checkJson(data["parameters"], name="MLP-8layers-dropout"):
                # header += "$"+str('{:.3f}'.format(data["parameters"]["l2_reg"])) + "$ & "
                row8+= "$"+str('{:.2f}'.format(getBestValAcc(data)*100))+"\pm"+str('{:.2f}'.format(confidenceInterval(getBestValAcc(data)*100, 10_000))) + "$ &"
    
                
print(header[0:-1])
print(row2[0:-1]+ " \\")
print(row4[0:-1]+ " \\")
print(row8[0:-1]+ " \\")

## Regularization

In [ ]:
datas = [[None for _ in range(3)] for _ in range(10)]

for subdir, dirs, files in os.walk(rootdir):
    for dir in sorted(dirs):
        if (dir!=".ipynb_checkpoints"):
            f = open(rootdir+dir+"/experiment.json")
            data = json.load(f)
            
            print(dir, data["parameters"]["name_model"])
            #if "09_" in dir:
            #    continue
            
            i,j = -1,-1
            if "MLP" in data["parameters"]["name_model"]: 
                if "2" in data["parameters"]["name_model"]:
                    i = 0
                elif "4" in data["parameters"]["name_model"]:
                    i = 1
                elif "8" in data["parameters"]["name_model"]:
                    i = 2

            if data["parameters"]["batch_norm"] and data["parameters"]["data_aug"]>0:
                j = 9
                print("y")
            elif "l2_reg" in data["parameters"].keys() and data["parameters"]["l2_reg"]>0:
                if data["parameters"]["l2_reg"]==0.001:
                    j = 0
                elif data["parameters"]["l2_reg"]==0.002:
                    j = 1
                elif data["parameters"]["l2_reg"]==0.005:
                    j = 2
                elif data["parameters"]["l2_reg"]==0.010:
                    j = 3
            elif data["parameters"]["dropout"]>0:
                if data["parameters"]["dropout"]==0.10:
                    j = 4
                elif data["parameters"]["dropout"]==0.25:
                    j = 5
                elif data["parameters"]["dropout"]==0.50:
                    j = 6
            elif data["parameters"]["batch_norm"]:
                j = 7
            elif data["parameters"]["data_aug"]:
                j = 8
                           
            #print(dir, i, j)    
            if i!=-1 and j!=-1:
                acc = '{:.2f}'.format(getBestValAcc(data)*100)
                ci = '{:.2f}'.format(confidenceInterval(getBestValAcc(data)*100, 10_000))
            
                datas[j][i]={"acc":acc, "ci":ci}
    
for row in datas:
    r = ""
    for col in row:
        #print(row,col)
        if col is not None:
            r += "$"+col["acc"]+"\pm"+col["ci"]+ "$ &"
    print(r[0:-1]+ "\\\\")
                


In [ ]:
fig, axs = plt.subplots(4,3,figsize=(20,16))
for ax in axs:
    for a in ax:
        a.set_axis_off()

for subdir, dirs, files in os.walk(rootdir):
    for dir in sorted(dirs):
        if (dir!=".ipynb_checkpoints"):
            f = open(rootdir+dir+"/experiment.json")
            data = json.load(f)
            if "4" in data["parameters"]["name_model"]:
                
                if data["parameters"]["batch_norm"] and data["parameters"]["data_aug"]>0:
                    img = mpimg.imread(rootdir+dir+"/accuracy.png")
                    axs[2,2].set_title("All", fontsize="xx-large")
                    axs[2,2].imshow(img)
                    
                    img = mpimg.imread(rootdir+dir+"/loss.png")
                    axs[3,2].imshow(img)
                elif "l2_reg" in data["parameters"].keys() and data["parameters"]["l2_reg"]==0.001:
                    img = mpimg.imread(rootdir+dir+"/accuracy.png")
                    axs[0,1].imshow(img)
                    axs[0,1].set_title("L2 Regularization", fontsize="xx-large")
                    
                    img = mpimg.imread(rootdir+dir+"/loss.png")
                    axs[1,1].imshow(img)
                elif data["parameters"]["dropout"]==0.25:
                    img = mpimg.imread(rootdir+dir+"/accuracy.png")
                    axs[0,2].set_title("Dropout  (dropout=0.25)", fontsize="xx-large")
                    axs[0,2].imshow(img)
                    
                    
                    img = mpimg.imread(rootdir+dir+"/loss.png")
                    axs[1,2].imshow(img)
        
                elif data["parameters"]["batch_norm"]:
                    img = mpimg.imread(rootdir+dir+"/accuracy.png")
                    axs[2,0].set_title("Batch Normalization", fontsize="xx-large")
                    axs[2,0].imshow(img)
                    
                    img = mpimg.imread(rootdir+dir+"/loss.png")
                    axs[3,0].imshow(img)
                elif data["parameters"]["data_aug"]:
                    img = mpimg.imread(rootdir+dir+"/accuracy.png")
                    axs[2,1].set_title("Data Augmentation", fontsize="xx-large")
                    axs[2,1].imshow(img)
                    
                    img = mpimg.imread(rootdir+dir+"/loss.png")
                    axs[3,1].imshow(img)
                elif data["parameters"]["name_model"]=="MLP-4layers" and data["parameters"]["initial_lr"]==0.01:
                    img = mpimg.imread(rootdir+dir+"/accuracy.png")
                    axs[0,0].set_title("None", fontsize="xx-large")
                    axs[0,0].imshow(img)
                    
                    img = mpimg.imread(rootdir+dir+"/loss.png")
                    axs[1,0].imshow(img)